In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [25]:
# Import dependencies
from bson.decimal128 import Decimal128

from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

#### The JSON file is formatted as an array, but mongoimport expects a series of individual JSON objects instead.

- Reformat your JSON file so that it contains newline-delimited JSON objects. This format is sometimes referred to as JSONL or NDJSON.

In [5]:
import json

# Replace 'establishments.json' with your JSON file path
input_filepath = 'Resources/establishments.json'

# Output file path
output_filepath = 'Resources/establishments.ndjson'

with open(input_filepath, 'r', encoding='utf-8') as infile, open(output_filepath, 'w') as outfile:
    data = json.load(infile)
    for entry in data:
        json.dump(entry, outfile)
        outfile.write('\n')

#### Use the following command to import the JSON file into a new MongoDB database named "uk_food" and a new collection named "establishments":

- Make sure mongodb tools are installed properly on your system
- Run the terminal or command prompt as an administrator
- mongoimport --db uk_food --collection establishments --file establishments.json


In [ ]:
mongoimport --db uk_food --collection establishments --file establishments.json

In [6]:
# confirm that our new database was created

# Get a list of all database names
db_list = mongo.list_database_names()
db_list

['admin', 'config', 'local', 'social_network_db', 'uk_food']

In [7]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [8]:
# review the collections in our new database
collection_list = db.list_collection_names()
collection_list

['establishments']

In [9]:
# Get a reference to the establishments collection
establishments_collection = db['establishments']

In [46]:
# review a document in the establishments collection
document = establishments_collection.find_one()
pprint(document)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65059ea2f548a69ecdf7004e'),
 'geocode': {'latitude': Decimal128('51.086058'),
             'longitude': Decimal128('1.196408')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta

In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

#### 1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [30]:
# Create a dictionary for the new restaurant data
penang_flavours = {
     "BusinessName":"Penang Flavours",
     "BusinessType":"Restaurant/Cafe/Canteen",
     "BusinessTypeID":"",
     "AddressLine1":"Penang Flavours",
     "AddressLine2":"146A Plumstead Rd",
     "AddressLine3":"London",
     "AddressLine4":"",
     "PostCode":"SE18 7DY",
     "Phone":"",
     "LocalAuthorityCode":"511",
     "LocalAuthorityName":"Greenwich",
     "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
     "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
     "scores":{
         "Hygiene":"",
         "Structural":"",
         "ConfidenceInManagement":""
     },
     "SchemeType":"FHRS",
     "geocode":{
         "longitude":"0.08384000",
         "latitude":"51.49014200"
     },
     "RightToReply":"",
     "Distance":4623.9723280747176,
     "NewRatingPending":True
}

In [31]:
# Insert the new restaurant into the collection
db.establishments.insert_one(penang_flavours)

In [32]:
# Check that the new restaurant was inserted
result = db.establishments.find_one({"BusinessName": "Penang Flavours"})

if result:
    print("The new restaurant was successfully inserted with the following details:")
    pprint(result)
else:
    print("The restaurant was not found in the database.")

The new restaurant was successfully inserted with the following details:
{'AddressLine1': '123 Flavour Street',
 'AddressLine2': 'Aroma Block',
 'AddressLine3': 'Greenwich',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': None,
 'FHRSID': 999999,
 'LocalAuthorityCode': '123',
 'LocalAuthorityEmailAddress': 'contact@greenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich City Council',
 'LocalAuthorityWebSite': 'https://www.greenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '1234567890',
 'PostCode': 'GR12345',
 'RatingDate': None,
 'RatingKey': 'fhrs_not_rated_en-gb',
 'RatingValue': None,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6505a178b3899b0c06d83ba5'),
 'geocode': {'latitude': Decimal128('51.4831'),
             'longitude': Decimal128('0.0134')},
 'links': [{'href': 'https://www.penangflavours.co.uk', 'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate'

#### 2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [33]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = db.establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"}, 
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)

# Since the query can return multiple documents, we use a loop to print each one
result

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

#### 3. Update the new restaurant with the `BusinessTypeID` you found.

In [34]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {"BusinessName": "Penang Flavours"}, 
    {"$set": {"BusinessTypeID": 1}}
)

In [45]:
# Confirm that the new restaurant was updated
updated_restaurant = db.establishments.find_one(
    {"BusinessName": "Penang Flavours"}, 
)

pprint(updated_restaurant)

{'AddressLine1': '123 Flavour Street',
 'AddressLine2': 'Aroma Block',
 'AddressLine3': 'Greenwich',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': None,
 'FHRSID': 999999,
 'LocalAuthorityCode': '123',
 'LocalAuthorityEmailAddress': 'contact@greenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich City Council',
 'LocalAuthorityWebSite': 'https://www.greenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '1234567890',
 'PostCode': 'GR12345',
 'RatingDate': None,
 'RatingKey': 'fhrs_not_rated_en-gb',
 'RatingValue': None,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6505a178b3899b0c06d83ba5'),
 'geocode': {'latitude': Decimal128('51.4831'),
             'longitude': Decimal128('0.0134')},
 'links': [{'href': 'https://www.penangflavours.co.uk', 'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '2023-09-16T00:00:00',
          'itemCount': 0,
          'pageNumber'

#### 4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [36]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f'There are {dover_count} documents with the LocalAuthorityName "Dover".')

There are 0 documents with the LocalAuthorityName "Dover".


In [37]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db.establishments.delete_many({"LocalAuthorityName": "Dover"})

# Print the number of documents deleted
print(f'Deleted {delete_result.deleted_count} documents with the LocalAuthorityName "Dover".')

Deleted 0 documents with the LocalAuthorityName "Dover".


In [38]:
# Check if any remaining documents include Dover
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

# Print the count of documents with LocalAuthorityName "Dover"
if dover_count == 0:
    print("No documents found with the LocalAuthorityName 'Dover'.")
else:
    print(f'Found {dover_count} documents with the LocalAuthorityName "Dover".')

No documents found with the LocalAuthorityName 'Dover'.


In [44]:
# Check that other documents remain with 'find_one'
one_document = db.establishments.find_one()

# Print the found document
if one_document:
    print("Found a document in the collection:\n")
    pprint(one_document)
else:
    print("No documents found in the collection.")

Found a document in the collection:

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65059ea2f548a69ecdf7004e'),
 'geocode': {'latitude': Decimal128('51.086058'),
             'longitude': Decimal128('1.196408')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177'

#### 5. Some of the number values are stored as strings, when they should be stored as numbers.

- Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [40]:
# Change the data type from String to Decimal for longitude and latitude

# Find all documents with latitude and longitude as strings
documents = db.establishments.find({'geocode.latitude': {'$type': 'string'}, 'geocode.longitude': {'$type': 'string'}})

# Loop through all the documents and update the latitude and longitude fields to decimal
for doc in documents:
    latitude = Decimal128(str(doc['geocode']['latitude']))
    longitude = Decimal128(str(doc['geocode']['longitude']))

    # Update the document with the new decimal latitude and longitude
    db.establishments.update_one({'_id': doc['_id']}, {'$set': {'geocode.latitude': latitude, 'geocode.longitude': longitude}})

- Use `update_many` to convert `RatingValue` to integer numbers.

In [41]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [42]:
# Change the data type from String to Integer for RatingValue

# Find all documents with RatingValue as strings and not in non_ratings list
documents = db.establishments.find({'RatingValue': {'$type': 'string', '$nin': non_ratings}})

for doc in documents:
    try:
        # Convert RatingValue to integer
        rating_value = int(doc['RatingValue'])

    except ValueError:
        # If conversion fails, set rating_value to None
        rating_value = None

    # Update the document with the new integer RatingValue
    db.establishments.update_one({'_id': doc['_id']}, {'$set': {'RatingValue': rating_value}})

In [43]:
# Check that the coordinates and rating value are now numbers

# Fetch a sample document
document = db.establishments.find_one()

# Print the data types of the fields
if document:
    latitude = document.get('geocode', {}).get('latitude')
    longitude = document.get('geocode', {}).get('longitude')
    rating_value = document.get('RatingValue')
    
    print(f"Data type of 'latitude': {type(latitude)}")
    print(f"Data type of 'longitude': {type(longitude)}")
    print(f"Data type of 'RatingValue': {type(rating_value)}")

else:
    print("No document found")

Data type of 'latitude': <class 'bson.decimal128.Decimal128'>
Data type of 'longitude': <class 'bson.decimal128.Decimal128'>
Data type of 'RatingValue': <class 'int'>
